In [1]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata

In [4]:
client = OpenAI(api_key=userdata.get('openai_api'))

In [5]:
# Fetch Text content from wikipedia
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBOT/0.0', 'en')
doc = wiki.page('Sri_Lanka').text
paragraphs = doc.split('\n\n') # chunk

In [6]:
import textwrap

In [7]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")

-----------------------------------------------------------------
Sri Lanka, historically known as Ceylon and officially the Democratic Socialist Republic of Sri
Lanka, is an island country in South Asia. It lies in the Indian Ocean, southwest of the Bay of
Bengal, separated from the Indian peninsula by the Gulf of Mannar and the Palk Strait. It shares a
maritime border with the Maldives in the southwest and India in the northwest. Sri Lanka has a
population of approximately 22 million and is home to many cultures, languages and ethnicities. The
Sinhalese people form the majority of the population, followed by the Sri Lankan Tamils, who are the
largest minority group and are concentrated in northern Sri Lanka; both groups have played an
influential role in the island's history. Other long-established groups include the Moors, Indian
Tamils, Burghers, Malays, Chinese, and Vedda. Sri Lanka's documented history goes back 3,000 years,
with evidence of prehistoric human settlements dating b

# Embedding model

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [9]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [10]:
docs_embed.shape

(52, 768)

In [11]:
docs_embed[0]

array([-6.60415366e-03, -1.40418913e-02,  6.20624945e-02,  3.26026119e-02,
        7.40868449e-02, -2.60289218e-02,  4.04493585e-02,  2.74554845e-02,
        3.79292332e-02, -3.90027394e-03,  1.79546084e-02,  2.94366740e-02,
        8.30710586e-03, -7.36194104e-03,  5.30408062e-02, -2.09762622e-02,
        8.83167312e-02,  3.50557752e-02,  1.14191305e-02, -4.65999125e-03,
        3.80281638e-03,  1.05417632e-02,  2.42362893e-03,  1.24024969e-04,
        5.67326695e-03, -1.67579986e-02, -2.67366450e-02,  1.94378756e-02,
       -2.09293775e-02, -7.89263926e-04, -2.73200832e-02, -1.88977213e-03,
        5.10002999e-03, -2.04790495e-02,  1.18852397e-02, -7.77046056e-03,
        9.19945352e-03,  4.99264710e-02, -1.78881157e-02, -3.95874456e-02,
       -7.16379061e-02, -2.84051504e-02, -1.15423184e-02,  1.20989354e-02,
        4.70048673e-02, -1.90816578e-02,  3.16767246e-02, -1.70213748e-02,
        3.92294955e-03, -2.65879240e-02,  8.54890328e-03,  3.67775895e-02,
       -9.35155991e-03, -

# Embed the Query

In [12]:
query = "What are some alternative names for Sri Lanka?"
query_embed = model.encode(query, normalize_embeddings=True)

In [13]:
query_embed.shape

(768,)

# Find the Closest Paragraphs to the Query

In [14]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [15]:
similarities.shape

(52,)

In [16]:
similarities

array([0.66756797, 0.7150059 , 0.49063176, 0.502255  , 0.5770285 ,
       0.4565762 , 0.47897792, 0.6030559 , 0.4374525 , 0.5021705 ,
       0.438964  , 0.4079057 , 0.5168574 , 0.48342767, 0.47339922,
       0.4601774 , 0.4307744 , 0.56656754, 0.39545235, 0.41656613,
       0.4615399 , 0.4467173 , 0.45392677, 0.500411  , 0.4980569 ,
       0.43423447, 0.5119715 , 0.5652287 , 0.45622432, 0.48731124,
       0.5000935 , 0.5455747 , 0.5125921 , 0.4055878 , 0.45317903,
       0.4271076 , 0.47469497, 0.47904304, 0.47649044, 0.4239917 ,
       0.50555384, 0.4808855 , 0.44261482, 0.54582226, 0.5013534 ,
       0.55382407, 0.5060072 , 0.37053588, 0.41780496, 0.35157844,
       0.43742907, 0.40217978], dtype=float32)

In [17]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [18]:
top_3_idx

[1, 0, 7]

In [19]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [20]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Toponymy In antiquity, Sri Lanka was known to travellers by a variety of names. According to the
Mahāvaṃsa, the legendary Prince Vijaya named the island Tambapaṇṇĩ ("copper-red hands" or "copper-
red earth"), because his followers' hands were reddened by the red soil of the area where he landed.
In Hindu mythology, the term Lankā ("Island") appears but it is unknown whether it refers to the
modern-day state. The Tamil term Eelam (Tamil: ஈழம், romanized: īḻam) was used to designate the
whole island in Sangam literature. The island was known under Chola rule as Mummudi Cholamandalam
("realm of the three crowned Cholas"). Ancient Greek geographers called it Taprobanā (Ancient Greek:
Ταπροβανᾶ) or Taprobanē (Ταπροβανῆ) from the word Tambapanni. The Persians and Arabs referred to it
as Sarandīb (the origin of the word "serendipity") from Sanskrit Siṃhaladvīpaḥ. Ceilão, the name
given to Sri Lanka by the Portuguese Empire when

In [22]:
query = "What are some alternative names for Sri Lanka?"

In [23]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [24]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [25]:
print(response.choices[0].message.content)

Some alternative names for Sri Lanka mentioned in the context are:

1. Tambapaṇṇĩ ("copper-red hands" or "copper-red earth")
2. Lankā
3. Eelam (Tamil: ஈழம், romanized: īḻam)
4. Mummudi Cholamandalam ("realm of the three crowned Cholas")
5. Taprobanā (Ancient Greek: Ταπροβανᾶ) or Taprobanē (Ταπροβανῆ)
6. Sarandīb
7. Ceilão
8. Ceylon

Additionally, the country is officially known as the Democratic Socialist Republic of Sri Lanka.
